# 使用 Unsloth 对 DeepSeek-R1-Distill-Qwen-1.5B 模型进行 LoRA 微调

本 Notebook 展示了如何使用 `unsloth` 库对 `deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B` 模型进行高效的 QLoRA (Low-Rank Adaptation) 微调。

整个流程包括：
1.  环境准备与库导入
2.  加载预训练模型和分词器 (Tokenizer)。
3.  在微调前，对模型进行简单的推理测试。
4.  下载和格式化训练数据集
5.  使用 `unsloth` 的 `FastLanguageModel` 来为模型添加 LoRA 适配器。
6.  配置 `SFTTrainer` 监督微调训练配置。
7.  启动训练，并观察 Loss 变化情况
8.  保存微调后的模型
9.  测试训练后的生成结果

### 1. 环境准备与库导入

首先，我们需要安装并导入所有必要的库。`transformers` 用于加载模型和分词器，`unsloth` 用于高效微调，`trl` 提供了 `SFTTrainer`，而 `datasets` 用于处理数据。

**注意**: 在运行此 Notebook 之前，请确保已安装所有依赖包：
```bash
pip install -r requirements.txt
```

In [1]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, DataCollatorForSeq2Seq
from datasets import Dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


W0901 11:54:47.222000 1684 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


🦥 Unsloth Zoo will now patch everything to make training faster!


### 2. 加载预训练模型和分词器 (Tokenizer)

In [2]:
# 定义模型和一些基本参数
max_seq_length = 8192
dtype = None # None 表示自动选择 (Float16 a T4, V100, BFloat16 a Ampere)
load_in_4bit = True # 使用 4bit 量化加载

# 这是您的模型标识符，请替换为您正在使用的模型
# 例如："qwen-1.5b_lora_model"
# model_name = "qwen-1.5b_lora_model" 
# model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B" 
model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit" 

# 这一步会返回一个经过 Unsloth 优化的模型和一个分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

D:\ProgramData\miniconda3\envs\deepseek\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:339: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:35.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.8.10: Fast Qwen2 patching. Transformers: 4.55.2.
   \\   /|    NVIDIA GeForce RTX 2080 Ti. Num GPUs = 1. Max memory: 11.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.8.0+cu129. CUDA: 7.5. CUDA Toolkit: 12.9. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


### 3. 微调前推理测试

在对模型进行任何修改之前，我们先用它来生成一段文本，看看原始模型的表现如何。这可以作为我们微调效果的基准参考。

In [3]:
# 模型推理的 Prompt 模板
inference_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>{}
"""

In [4]:
FastLanguageModel.for_inference(model)

question = "男，28岁，程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。"

inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")
attention_mask = inputs.input_ids.ne(tokenizer.pad_token_id).long().to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

In [5]:
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [6]:
print(response[0].split("### Response:")[1])


<think>
好，我来仔细分析一下这个问题。用户是一位28岁的程序员，最近一周每天工作到半夜，导致头晕、脖子疼和恶心。首先，头晕和脖子疼可能与神经功能有关，也有可能是过度劳累或紧张引起的。恶心也有可能是消化系统的问题，或者是精神因素导致的。需要考虑是否有其他健康问题，比如高血压、糖尿病、心脏病等，因为这些可能导致头晕和恶心。此外，长时间工作可能影响到内分泌系统，比如激素变化，也可能影响到心脏功能。需要考虑是否有其他因素，比如压力、睡眠质量、饮食、运动等。建议用户进行详细的身体检查，包括血压、血糖、心率、呼吸等指标，以排除潜在的健康问题。如果身体没有问题，可能需要考虑药物治疗，比如抗恶心药物、抗抑郁药物等，以缓解症状。如果情况没有改善，可能需要进行进一步的诊断，比如神经系统评估。如果问题持续或加重，可能需要就医进行治疗。需要建议用户尽快联系医生，以便获得专业的诊断和治疗建议。
</think>

男，28岁，程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。

### 分步思考过程：
1. **理解问题**：用户是一名程序员，工作时间长期超过12小时，导致头晕、脖子疼和恶心。这些症状可能与神经功能、内分泌系统、呼吸系统、消化系统或其他健康问题有关。
2. **初步判断**：头晕、脖子疼和恶心可能提示神经功能异常或长期压力。恶心可能与消化系统或内分泌系统有关。
3. **考虑其他健康问题**：可能涉及高血压、糖尿病、心脏病、甲状腺功能亢进、肾上腺皮质醇缺乏症等，这些都可能影响工作状态。
4. **建议检查身体**：建议进行血压、血糖、心率、呼吸、消化功能等指标的详细检查，排除潜在健康问题。
5. **药物治疗**：根据检查结果，考虑使用抗恶心、抗抑郁、抗焦虑药物等进行治疗。
6. **进一步诊断**：如果情况没有改善或加重，可能需要进行神经科或内分泌科的诊断。
7. **建议就医**：如果以上措施无效，建议用户尽快联系专业医生进行进一步诊断和治疗。

### 最终回复：
请立即联系医生，详细检查身体状况，包括血压、血糖、心率、呼吸、消化功能等指标。根据检查结果，评估可能的健康问题并考虑药物治疗。如果症状持续或加重，建议寻求进一步的医疗评估和治疗建议。


---

### 4. 下载和格式化训练数据集


医学推理数据集：https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/viewer/zh

![dataset](images/dataset.png)

In [7]:
# 模型训练的 Prompt 模板
train_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>
{}
</think>
{}
"""

In [8]:
EOS_TOKEN = tokenizer.eos_token # 添加 EOS Token

def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        # 将 EOS Token 添加到样本最后
        text = train_prompt.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "zh", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [9]:
dataset[0]["text"]

'以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。\n请撰写一份恰当的回复，以完成该请求。\n在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。\n\n\n### Instruction:\n你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。\n请回答以下医学问题。\n\n### Question:\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### Response:\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？\n\n再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。\n\n看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。\n\n不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。\n\n

In [10]:
from IPython.display import display, Markdown

display(Markdown(dataset[0]["text"]))

以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？

### Response:
<think>
这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。

用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。

但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。

等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。

仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。

回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？

再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。

看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。

不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。

哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。
</think>
从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的儿童身上。建议结合中医的清热解毒、祛湿消肿的治疗方法进行处理，并配合专业的医疗建议进行详细诊断和治疗。
<｜end▁of▁sentence｜>

### 5. 使用 Unsloth 添加 LoRA 适配器

这是使用 `unsloth` 的核心步骤。我们调用 `FastLanguageModel.get_peft_model`，它会非常高效地为模型注入 LoRA 模块。

- `r`: LoRA 的秩 (rank)，是控制模型复杂度和参数量的关键超参数。
- `target_modules`: 指定要在哪些线性层（如注意力机制的 q, k, v, o 投影层）上应用 LoRA。
- `lora_alpha`: LoRA 的缩放因子，通常设置为 `r` 的两倍或与 `r` 相同。
- `use_gradient_checkpointing`: 一种节省显存的技术，对于训练大模型至关重要。

In [11]:
# 因为 `model` 对象现在是由 Unsloth 创建的，它包含了所有必需的属性
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
      "q_proj",
      "k_proj",
      "v_proj",
      "o_proj",
      "gate_proj",
      "up_proj",
      "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=1432,
    use_rslora=False,
    loftq_config=None,
)
# 检查模型结构，确认 LoRA 适配器已添加
print(model)

Unsloth 2025.8.10 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536, padding_idx=151654)
        (layers): ModuleList(
          (0): Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
       

### 6. 配置 SFTTrainer

`SFTTrainer` (Supervised Fine-tuning Trainer) 是一个专门用于指令微调的训练器。我们需要配置 `TrainingArguments` 来指定所有的训练参数，如批量大小、学习率、优化器等。

In [12]:
from trl import SFTConfig, SFTTrainer
import sys
import os

# 手动添加 Unsloth 编译缓存目录到 Python 路径
# 注意：路径需替换为你实际的 unsloth_compiled_cache 路径（从错误日志中获取）
cache_dir = "D:\\workspace\\deepseek-quickstart\\distill\\unsloth_compiled_cache"
if cache_dir not in sys.path:
    sys.path.append(cache_dir)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 32,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1432,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

### 7. 开始训练

一切准备就绪后，调用 `trainer.train()` 即可开始微调过程。训练结束后，会返回包含训练统计信息（如训练损失）的对象。

In [13]:
# 1. 先导入必要库，再设置环境变量（必须放在代码最开头！）
import os
os.environ["TORCHINDUCTOR_CACHE_DIR"] = "D:\\torch_temp"  # 自定义临时目录（用绝对路径，双反斜杠）
os.environ["TRITON_CACHE_DIR"] = "D:\\torch_temp\\triton"  # Triton 单独缓存目录

# 2. 确保目录存在（不存在则创建）
if not os.path.exists(os.environ["TORCHINDUCTOR_CACHE_DIR"]):
    os.makedirs(os.environ["TORCHINDUCTOR_CACHE_DIR"], exist_ok=True)
if not os.path.exists(os.environ["TRITON_CACHE_DIR"]):
    os.makedirs(os.environ["TRITON_CACHE_DIR"], exist_ok=True)

os.environ["CUDA_LAUNCH_BLOCKING"] = "0"  # 关闭同步阻塞（默认应该是0）
os.environ["TORCH_CUDNN_BENCHMARK"] = "1"  # 启用 cudnn 自动调优
os.environ["NCCL_DEBUG"] = "WARN"  # 仅输出 NCCL 警告，减少日志开销

torch.cuda.empty_cache()  # 清空未使用的GPU缓存

print("模型设备：", model.device)  # 应显示cuda:0

trainer.accelerator.state.use_cuda_graph = True



trainer_stats = trainer.train()

# 打印训练统计信息
print(trainer_stats)

模型设备： cuda:0


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 32 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (32 x 2 x 1) = 64
 "-____-"     Trainable parameters = 18,464,768 of 1,795,552,768 (1.03% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,entropy
1,3.218000,0
2,3.107100,No Log
3,3.168900,No Log
4,3.133900,No Log
5,2.998000,No Log
6,2.956800,No Log
7,2.951400,No Log
8,2.948200,No Log
9,2.894300,No Log
10,2.802400,No Log


TrainOutput(global_step=60, training_loss=2.470342234770457, metrics={'train_runtime': 1273.9381, 'train_samples_per_second': 3.014, 'train_steps_per_second': 0.047, 'total_flos': 3.631893877830451e+16, 'train_loss': 2.470342234770457, 'epoch': 0.1901743264659271})


### 8. 保存微调后的模型（Lora）

训练完成后，您可以再次进行推理，比较微调后的模型与原始模型的差异。如果对结果满意，可以使用 `model.save_pretrained("your_lora_adapter_path")` 来保存训练好的 LoRA 适配器。

In [14]:
model.save_pretrained("qwen-1.5b_lora_model")

In [15]:
tokenizer.save_pretrained("qwen-1.5b_lora_model")

('qwen-1.5b_lora_model\\tokenizer_config.json',
 'qwen-1.5b_lora_model\\special_tokens_map.json',
 'qwen-1.5b_lora_model\\chat_template.jinja',
 'qwen-1.5b_lora_model\\tokenizer.json')

In [16]:
# 模型保存方式二选一（要么使用上面的分开保存，要么使用这里的合并 Lora 保存）
# model.save_pretrained_merged("qwen-1.5b_lora_model", tokenizer, save_method="merged_16bit")

### 9. 测试训练后的生成结果

In [17]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

question="一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？", # Question
inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1000,
    use_cache=True,
)

In [18]:
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(output[0].split("### Response:")[1])


<think>
这个病人得了急性阑尾炎，已经跑了5天，感觉腹痛好像在慢慢减轻，但还是有一点发热。在检查时，右下腹里看到压痛的包块，这让我有点担心。压痛包块，这可能是阑尾炎的一种表现。

首先，我得想想阑尾炎的常见症状。急性阑尾炎通常会伴随腹痛、发热，甚至有压痛包块。这些症状在急性期更容易出现。所以，病人现在的情况符合急性阑尾炎的特点。

现在，病人已经到了急性期，而且症状已经有所缓解。但是，压痛包块的出现让我有些犹豫，因为压痛包块在急性期是常见的一种表现，而且在急性期的病人，压痛包块可能更多是伴随疼痛的。

我觉得，先检查一下压痛包块是否是阑尾炎的典型表现。如果发现压痛包块，我应该立即做进一步的检查，比如行影像学检查，比如超声或者CT扫描，以确认是否确实有压痛包块。同时，也要注意是否有其他症状，比如发热。

在做这些检查之前，我得考虑病人是否需要立即处理，比如是否需要手术。如果有压痛包块，可能需要手术切除，以防止进一步的感染和损伤。但是，如果症状已经有所缓解，或者病人有其他症状，比如发热，我们可能需要谨慎处理。

在病人没有明显症状的情况下，我们通常会建议进行影像学检查，以确认压痛包块的存在。如果有压痛包块，可以考虑手术切除。不过，如果病人没有其他症状，可能需要考虑在急性期避免手术。

如果病人有其他症状，比如发热，我们应该避免手术，但要同时注意其风险。因为急性期的病人如果出现发热，可能有感染的风险，所以手术可能还是建议的。

总的来说，病人的情况需要根据症状和检查结果来决定下一步。如果病人有压痛包块，最好立即处理，但如果有其他症状，可能需要谨慎处理。
</think>
根据病人的情况，病人已经确诊为急性阑尾炎，并且已经到了急性期。在急性期，压痛包块是一种常见表现，通常伴随腹痛、发热等。根据临床经验，如果病人在急性期或急性期已经缓解症状，可以考虑进行影像学检查以确认压痛包块的存在。如果发现压痛包块，应立即进行进一步的检查，如超声或CT扫描，以确认其存在。如果发现压痛包块，可以考虑手术切除以防止进一步的感染和损伤。然而，如果病人在急性期或急性期症状已经缓解，且没有其他症状，可以采取谨慎处理，以防止感染风险。因此，应根据具体检查结果和病人的症状情况来决定下一步处理措施。



In [19]:
def generate_response(question: str, model, tokenizer, inference_prompt: str, max_new_tokens: int = 1024) -> str:
    """
    使用指定的模型和分词器为给定的医学问题生成响应。

    Args:
        question (str): 需要模型回答的医学问题。
        model: 已加载的 Unsloth/Hugging Face 模型。
        tokenizer: 对应的分词器。
        inference_prompt (str): 用于格式化输入的 f-string 模板。
        max_new_tokens (int, optional): 生成响应的最大 token 数量。默认为 1024。

    Returns:
        str: 模型生成的响应文本，已去除 prompt 部分。
    """
    # 1. 使用模板格式化输入
    prompt = inference_prompt.format(
        question, # 填充问题
        "",       # 留空，让模型生成 CoT 和 Response
    )

    # 2. 将格式化后的 prompt 进行分词，并转移到 GPU
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    # 3. 使用模型生成输出
    # use_cache=True 用于加速解码过程
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        use_cache=True,
    )
    
    # 4. 将生成的 token 解码为文本
    # skip_special_tokens=True 会移除像 EOS_TOKEN 这样的特殊标记
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # 5. 切分字符串，只返回 "### Response:" 之后的部分
    # 使用 .split() 分割并获取响应内容，.strip() 用于去除可能存在的前后空白字符
    response_part = decoded_output.split("### Response:")
    if len(response_part) > 1:
        return response_part[1].strip()
    else:
        # 如果模型没有生成 "### Response:" 标记，则返回整个生成内容以供调试
        return decoded_output

In [20]:
my_question = "对于一名60岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，请问哪一项实验室检查对了解胸水的性质更有帮助？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
嗯，这位60岁男性患者在X线检查中发现右侧肋膈角消失，这让我想到肺结核。肺结核通常会导致胸腔积液，尤其是右侧。不过，胸腔积液的性质其实还需要进一步了解，才能准确判断是肺结核还是其他原因造成的。所以，我想到了几项可能对胸腔积液性质有帮助的实验室检查。

首先是血常规，它能告诉我们血细胞的含量，这有助于了解是否发生感染引起的白细胞增多。接着是胸腔CT检查，这能更详细地看到胸腔的结构，比如看到更多的积液，这对我理解积液的性质很有帮助。

然后是血液样本，比如血液酸碱度、血钙含量，这些信息可以告诉我们血钙是否过高，这在肺结核中是重要的，因为它会直接影响钙吸收。

再看看胸腔CT，它能提供3D图像，帮助我更好地理解积液的分布和形态。然后是血液样本中的钙质分析，这对了解钙吸收的状况有帮助。

最后，胸腔CT的X线检查也能直接帮助我确定胸腔积液的性质，比如是否是气囊积液还是肺结核引起的。这对我来说非常重要，因为不同的性质需要不同的处理。

综合这些信息，我觉得血液样本和胸腔CT检查都是必要的，因为它们能提供最详细的信息。我得先查这些样本，看看血钙含量是否过高，然后用CT图进一步确认。这可能有助于更准确地诊断出是肺结核还是其他原因导致的胸腔积液。
</think>
为了进一步了解胸腔积液的性质，需要进行以下实验室检查：

1. **血液样本分析**：
   - 测量血钙含量，以判断血钙是否过高，这在肺结核中至关重要，因为肺结核会导致钙吸收不足。

2. **胸腔CT检查**：
   - 通过3D图像确认胸腔积液的分布，明确是气囊积液还是肺结核导致的。

3. **血液样本中的钙质分析**：
   - 进行钙质分析，以评估钙吸收情况，帮助排除肺结核的可能。

4. **胸腔CT检查**：
   - 使用X线检查明确胸腔积液的性质，例如气囊积液或肺结核引起的肺结核性胸腔积液。

这些检查可以帮助更准确地诊断出是肺结核还是其他原因引起的胸腔积液，以确保治疗方案的正确性。


In [21]:
my_question = "对于一名 28 岁的男性患者，工作是程序员，常年熬夜，最近突然感觉头晕目眩，甚至有点恶心。请问有可能是什么疾病？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
患者28岁，工作程序员，长时间熬夜，最近出现头晕目眩，还有轻微恶心。这让我想到可能的疾病。

首先，长时间熬夜导致头晕和恶心，这可能与某些慢性疾病有关。像肝炎，特别是肝炎C病毒引起的肝炎，可能和长期的睡眠不足有关。

肝炎C病毒引起的肝炎是个不错的选择，因为它通常和肝功能异常有关，比如肝细胞受损。这种疾病通常需要早期诊断，因为如果发现，治疗起来可能会更有效。

不过，患者最近出现的头晕和恶心，还不能排除其他可能性，比如脑炎、糖尿病、甲状腺功能障碍等。不过，考虑到患者的长期熬夜和症状，肝炎C病毒可能是一个更合适的解释。

接下来，想一下是否有其他可能的疾病，比如肝炎C病毒引起的肝炎，或者脑炎。但因为患者最近的症状是头晕和恶心，而这些症状通常与肝功能异常有关，所以我觉得肝炎C病毒更符合。

再进一步思考，这个患者长期熬夜，这可能与肝细胞损伤有关，而肝炎C病毒确实会导致肝细胞受损，特别是肝脏功能异常。因此，结合患者的症状和症状的可能来源，我觉得肝炎C病毒引起的肝炎可能是合理的解释。

所以，综合来看，患者最有可能患上肝炎C病毒引起的肝炎。
</think>
根据患者的症状和症状的可能来源，该患者28岁，工作程序员，长期熬夜，最近出现头晕目眩和轻微恶心，这种情况最有可能是由肝炎C病毒引起的肝炎。肝炎C病毒通常会导致肝细胞受损，特别是肝脏功能异常。患者长期的睡眠不足和出现的头晕目眩和恶心症状进一步支持了这种解释。因此，结合患者的长期症状和症状的可能来源，肝炎C病毒可能是合理的解释。
